In [0]:
# Docs
# https://spotipy.readthedocs.io/en/2.7.1/#
# https://console.firebase.google.com/project/spotify-f1cf5/database/firestore/data~2F

In [1]:
# Run once at start
!pip install spotipy

In [4]:
import spotipy
import numpy as np
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from google.colab import files


Saving spotify_credentials.json to spotify_credentials.json


In [0]:
# FIREBASE IMPLEMENTATION

#!pip install firebase
#!pip install firestore
# uploaded = files.upload() # upload cred
# import os
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/spotify-f1cf5-firebase-adminsdk-xxrd4-763e07c601.json'
#from google.cloud import firestore
#import firebase_admin
#from firebase_admin import credentials
# cred = credentials.Certificate('/content/spotify-f1cf5-firebase-adminsdk-xxrd4-763e07c601.json')
# firebase_admin.initialize_app(cred, {
#     'databaseURL': 'https://spotify-f1cf5.firebaseio.com'
# })

In [0]:
# Upload JSON of your credentials with "client_id" = {your client id}, "client_secret" = {your client secret}
uploaded = files.upload()
import json

with open('spotify_credentials.json') as json_file:
    creds = json.load(json_file)

In [0]:
client_credentials_manager = SpotifyClientCredentials(client_id=creds['client_id'], client_secret=creds['client_secret'])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [0]:
# First set of calls
artist_name = []
artist_id = []
album_name = []
album_id = []
track_name = []
track_id = []
track_pop = []
#track_year = []
track_spotify_genre = []

# Second set
key = []
acousticness = []
danceability = []
duration_ms = []
energy = []
instrumentalness = []
liveness = []
loudness = []
mode = []
speechiness = []
tempo = []
time_signature = []
valence = []

# Third set
artist_genre = []


In [0]:
# Get collection of song ids and basic track details - 50 tracks per call

for query in ['genre:classical', 'genre:country', 'genre:hip-hop', 'genre:house',
              'genre:indie','genre:pop','genre:r&b','genre:rock']:
#for query in ['year:2011', 'year:2012', 'year:2013', 'year:2014','year:2015', 'year:2016','year:2017', 'year:2018', 'year:2019', 'year:2020']: 
  for i in range(0,2000,50):
    track_results = sp.search(q=query, type='track', limit=50, offset=i) # change 'q=' argument for year
    for item in track_results['tracks']['items']:
      artist_name.append(item['album']['artists'][0]['name'])
      artist_id.append(item['album']['artists'][0]['id'])
      track_name.append(item['name'])
      track_id.append(item['id'])
      track_pop.append(item['popularity'])
      #track_year.append(query[5:])
      track_spotify_genre.append(query[6:])
      album_name.append(item['album']['name'])
      album_id.append(item['album']['id'])


In [0]:
# Get Audio Analysis (high level) track details  - 50 tracks per call

for i in range(0,len(track_id),50):
  track_features = sp.audio_features(track_id[i:i+50]) # returns features of first 50 tracks
  for j in range(0,len(track_features)): # iterate over those 50 tracks
    if track_features[j] is None:  # if track does not have audio features from spotify
      track_features[j] = {}
    key.append(track_features[j].get('key', np.nan))
    acousticness.append(track_features[j].get('acousticness', np.nan))
    danceability.append(track_features[j].get('danceability', np.nan))
    duration_ms.append(track_features[j].get('duration_ms', np.nan))
    energy.append(track_features[j].get('energy', np.nan))
    instrumentalness.append(track_features[j].get('instrumentalness', np.nan))
    liveness.append(track_features[j].get('liveness', np.nan))
    loudness.append(track_features[j].get('loudness', np.nan))
    mode.append(track_features[j].get('mode', np.nan))
    speechiness.append(track_features[j].get('speechiness', np.nan))
    tempo.append(track_features[j].get('tempo', np.nan))
    time_signature.append(track_features[j].get('time_signature', np.nan))
    valence.append(track_features[j].get('valence', np.nan))

retrying ...2secs


In [0]:
# Get artist genre - 50 tracks (artists) per call
for i in range(0,len(artist_id),50):
  artist_features = sp.artists(artist_id[i:i+50]) # returns features of next 50 tracks
  for j in range(0,len(artist_features['artists'])): # iterate over those 50 tracks
    if artist_features['artists'][j] is None:
      artist_features['artists'][j] = {}
    artist_genre.append(artist_features['artists'][j].get('genres', np.nan))

retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs


In [0]:
df = pd.DataFrame({"track_id":track_id,
                   "track_name":track_name,
                   "track_year":track_year,
                   #"track_spotify_genre":track_spotify_genre,
                   "art_name":artist_name,
                   "art_id":artist_id,
                   "art_genre":artist_genre,
                   "alb_name":album_name,
                   "alb_id":album_id,
                   "track_pop":track_pop,
                   "key":key,
                   "acousticness":acousticness,
                   "danceability":danceability,
                   "duration_ms":duration_ms,
                   "energy":energy,
                   "instrumentalness":instrumentalness,
                   "liveness":liveness,
                   "loudness":loudness,
                   "mode":mode,
                   "speechiness":speechiness,
                   "tempo":tempo,
                   "time_signature":time_signature,
                   "valence":valence
                   })
#track_dict = df.to_dict(orient='records')

In [0]:
df.to_csv('rebalance.csv')
files.download('rebalance.csv')

In [0]:
#for i, item in enumerate(track_dict):  
#  firestore.Client().collection('tracks').document(str(i)).set(item)